## run this cell before starting spark cluster connection (corrects module error)

In [1]:
# spark.jars.packages com.databricks:spark-avro_2.11:4.0.0
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/etc/spark/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [4]:
df_lookup = spark.read.format("com.databricks.spark.avro").load("/user/rucio01/dumps/2018-04-01/contents")
for i in ['02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18',
          '19','20','21','22','23','24','25','26','27','28','29','30']:
    df_lookup = df_lookup.union(spark.read.format("com.databricks.spark.avro").load("/user/rucio01/dumps/2018-04-"+ i + "/contents"))

#df_lookup.count()

23004415457

In [5]:
df_lookup.printSchema()

root
 |-- SCOPE: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- CHILD_SCOPE: string (nullable = true)
 |-- CHILD_NAME: string (nullable = true)
 |-- DID_TYPE: string (nullable = true)
 |-- CHILD_TYPE: string (nullable = true)
 |-- LENGTH: string (nullable = true)
 |-- BYTES: string (nullable = true)
 |-- ADLER32: string (nullable = true)
 |-- MD5: string (nullable = true)
 |-- RULE_EVALUATION: string (nullable = true)
 |-- UPDATED_AT: long (nullable = true)
 |-- CREATED_AT: long (nullable = true)
 |-- GUID: binary (nullable = true)
 |-- EVENTS: string (nullable = true)



In [6]:
df_lookup = df_lookup.dropDuplicates(['SCOPE','NAME','CHILD_SCOPE','CHILD_NAME','DID_TYPE'])

In [7]:
df_lookup.count()

827642445

In [10]:
df_lookup = df_lookup.drop('GUID')
df_lookup.write.csv('/user/sprenner/lookup.csv', sep='\t')